In [105]:
import os
import sys
import time
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
import scipy.io
from scipy.interpolate import griddata
from pyDOE import lhs

import torch
import torch.nn as nn
import torch.optim as optim

# Set random seeds for reproducibility
torch.manual_seed(1234)
np.random.seed(1234)

# Check for GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\033[91mUsing device: {device}\033[0m")



time.sleep(2)

Using device: cuda


In [106]:
class NeuralNetwork(nn.Module):
    def __init__(self, layers):
        super(NeuralNetwork, self).__init__()
        
        # Extract layer dimensions
        self.depth = len(layers) - 1
        
        # Define activation function
        self.activation = nn.Tanh()
        
        # Initialize layers
        layer_list = []
        for i in range(self.depth):
            layer_list.append(nn.Linear(layers[i], layers[i+1]))
            
            # Apply Xavier initialization
            nn.init.xavier_normal_(layer_list[i].weight)
            nn.init.zeros_(layer_list[i].bias)
            
        self.layers = nn.ModuleList(layer_list)
    
    def forward(self, x):
        for i in range(self.depth - 1):
            x = self.activation(self.layers[i](x))
        
        # Last layer without activation
        x = self.layers[-1](x)
        return x

In [107]:
class PhysicsInformedNN:
    def __init__(self, x0, u0, v0, tb, X_f, layers, lb, ub, X_u_train):
        
        # Convert numpy arrays to torch tensors
        self.x0 = torch.tensor(x0, dtype=torch.float32).to(device)
        self.t0 = torch.tensor(np.zeros_like(x0), dtype=torch.float32).to(device)
        self.u0 = torch.tensor(u0, dtype=torch.float32).to(device)
        self.v0 = torch.tensor(v0, dtype=torch.float32).to(device)
        
        # Boundary conditions
        self.x_lb = torch.tensor(lb[0] * np.ones_like(tb), dtype=torch.float32).to(device)
        self.t_lb = torch.tensor(tb, dtype=torch.float32).to(device)
        
        self.x_ub = torch.tensor(ub[0] * np.ones_like(tb), dtype=torch.float32).to(device)
        self.t_ub = torch.tensor(tb, dtype=torch.float32).to(device)
        
        # Collocation points
        self.x_f = torch.tensor(X_f[:, 0:1], dtype=torch.float32).to(device)
        self.t_f = torch.tensor(X_f[:, 1:2], dtype=torch.float32).to(device)
        
        # Store bounds
        self.lb = torch.tensor(lb, dtype=torch.float32).to(device)
        self.ub = torch.tensor(ub, dtype=torch.float32).to(device)
        
        # Additional data
        self.X_u_train = X_u_train
        
        # Initialize neural network
        self.layers = layers
        self.model = NeuralNetwork(layers).to(device)
        
        # Define optimizer
        self.optimizer_adam = optim.Adam(self.model.parameters(), lr=0.001)
        self.optimizer_lbfgs = optim.LBFGS(
            self.model.parameters(), 
            max_iter=20000, 
            max_eval=20000, 
            history_size=50, 
            tolerance_grad=1e-5, 
            tolerance_change=1.0 * np.finfo(float).eps, 
            line_search_fn="strong_wolfe"
        )
        
        # Store loss history
        self.loss_adam = []
        self.loss_lbfgs = []
        self.iteration = 0
    
    def net_uv(self, x, t):
        # Normalize inputs to [-1, 1]
        X = torch.cat([x, t], dim=1)
        X_normalized = 2.0 * (X - self.lb) / (self.ub - self.lb) - 1.0
        
        # Forward pass through the network
        uv = self.model(X_normalized)
        u, v = uv[:, 0:1], uv[:, 1:2]
        
        return u, v
    
    def net_f_uv(self, x, t):
        # Make x and t require grad for computing derivatives
        x = x.clone().detach().requires_grad_(True)
        t = t.clone().detach().requires_grad_(True)
        
        # Forward pass
        u, v = self.net_uv(x, t)
        
        # Compute gradients using autograd
        u_t = torch.autograd.grad(
            u, t, 
            grad_outputs=torch.ones_like(u),
            retain_graph=True,
            create_graph=True
        )[0]
        
        u_x = torch.autograd.grad(
            u, x,
            grad_outputs=torch.ones_like(u),
            retain_graph=True,
            create_graph=True
        )[0]
        
        u_xx = torch.autograd.grad(
            u_x, x,
            grad_outputs=torch.ones_like(u_x),
            retain_graph=True,
            create_graph=True
        )[0]
        
        v_t = torch.autograd.grad(
            v, t,
            grad_outputs=torch.ones_like(v),
            retain_graph=True,
            create_graph=True
        )[0]
        
        v_x = torch.autograd.grad(
            v, x,
            grad_outputs=torch.ones_like(v),
            retain_graph=True,
            create_graph=True
        )[0]
        
        v_xx = torch.autograd.grad(
            v_x, x,
            grad_outputs=torch.ones_like(v_x),
            retain_graph=True,
            create_graph=True
        )[0]
        
        # NLS equation residuals
        f_u = u_t + 0.5*v_xx + (u**2 + v**2)*v
        f_v = v_t - 0.5*u_xx - (u**2 + v**2)*u
        
        return f_u, f_v
    
    def loss_fn(self):
        # Initial condition loss
        u_pred0, v_pred0 = self.net_uv(self.x0, self.t0)
        loss_u0 = torch.mean((self.u0 - u_pred0)**2)
        loss_v0 = torch.mean((self.v0 - v_pred0)**2)
        
        # Boundary condition losses
        # Left boundary
        u_lb, v_lb = self.net_uv(self.x_lb, self.t_lb)
        
        # Right boundary
        u_ub, v_ub = self.net_uv(self.x_ub, self.t_ub)
        
        # Enforce periodicity
        loss_u_bnd = torch.mean((u_lb - u_ub)**2)
        loss_v_bnd = torch.mean((v_lb - v_ub)**2)
        
        # Periodicity of derivatives
        # Need to compute derivatives for boundary conditions
        x_lb = self.x_lb.clone().detach().requires_grad_(True)
        t_lb = self.t_lb.clone().detach().requires_grad_(True)
        u_lb, v_lb = self.net_uv(x_lb, t_lb)
        
        u_x_lb = torch.autograd.grad(
            u_lb, x_lb,
            grad_outputs=torch.ones_like(u_lb),
            retain_graph=True,
            create_graph=True
        )[0]
        
        v_x_lb = torch.autograd.grad(
            v_lb, x_lb,
            grad_outputs=torch.ones_like(v_lb),
            retain_graph=True,
            create_graph=True
        )[0]
        
        x_ub = self.x_ub.clone().detach().requires_grad_(True)
        t_ub = self.t_ub.clone().detach().requires_grad_(True)
        u_ub, v_ub = self.net_uv(x_ub, t_ub)
        
        u_x_ub = torch.autograd.grad(
            u_ub, x_ub,
            grad_outputs=torch.ones_like(u_ub),
            retain_graph=True,
            create_graph=True
        )[0]
        
        v_x_ub = torch.autograd.grad(
            v_ub, x_ub,
            grad_outputs=torch.ones_like(v_ub),
            retain_graph=True,
            create_graph=True
        )[0]
        
        loss_u_x_bnd = torch.mean((u_x_lb - u_x_ub)**2)
        loss_v_x_bnd = torch.mean((v_x_lb - v_x_ub)**2)
        
        # PDE residual loss
        f_u, f_v = self.net_f_uv(self.x_f, self.t_f)
        loss_f_u = torch.mean(f_u**2)
        loss_f_v = torch.mean(f_v**2)
        
        # Total loss
        loss = loss_u0 + loss_v0 + \
               loss_u_bnd + loss_v_bnd + \
               loss_u_x_bnd + loss_v_x_bnd + \
               loss_f_u + loss_f_v
        
        return loss
    
    def closure(self):
        self.optimizer_lbfgs.zero_grad()
        loss = self.loss_fn()
        loss.backward()
        self.iteration += 1
        if self.iteration % 100 == 0:
            print(f'Iteration {self.iteration}: Loss L-BFGS = {loss.item():.5e}')
        self.loss_lbfgs.append(loss.item())
        return loss
    
    def train(self, epochs):
        self.model.train()
        
        # First train with Adam
        start_time = time.time()
        for epoch in range(epochs):
            self.optimizer_adam.zero_grad()
            loss = self.loss_fn()
            loss.backward()
            self.optimizer_adam.step()
            
            self.loss_adam.append(loss.item())
            
            if epoch % 100 == 0:
                elapsed = time.time() - start_time
                print(f'Epoch {epoch}, Loss Adam: {loss.item():.5e}, Time: {elapsed:.2f}s')
                start_time = time.time()
        
        print('Finished Adam optimization. Starting L-BFGS...')
        
        # Then refine with L-BFGS
        self.iteration = 0
        self.optimizer_lbfgs.step(self.closure)
        
        print('Done!')
    
    def predict(self, X_star):
        self.model.eval()
        
        x = torch.tensor(X_star[:, 0:1], dtype=torch.float32).to(device)
        t = torch.tensor(X_star[:, 1:2], dtype=torch.float32).to(device)
        
        with torch.no_grad():
            u, v = self.net_uv(x, t)
        
        x_f = torch.tensor(X_star[:, 0:1], dtype=torch.float32).to(device).requires_grad_(True)
        t_f = torch.tensor(X_star[:, 1:2], dtype=torch.float32).to(device).requires_grad_(True)
        
        with torch.no_grad():
            f_u, f_v = self.net_f_uv(x_f, t_f)
        
        return u.cpu().numpy(), v.cpu().numpy(), f_u.cpu().numpy(), f_v.cpu().numpy()

In [108]:
# Load the data from NLS.mat (formato de PINNs_Torch)
try:
    data = scipy.io.loadmat('NLS.mat')
    print("Data loaded successfully!")
except:
    print("Error loading data file. Make sure 'NLS.mat' is in the current directory.")

# Extract data
t = data['tt'].flatten()[:, None]
x = data['x'].flatten()[:, None]
Exact = data['uu']

Exact_u = np.real(Exact)
Exact_v = np.imag(Exact)
Exact_h = np.sqrt(Exact_u ** 2 + Exact_v ** 2)

# Create mesh for prediction
X, T = np.meshgrid(x, t)
X_star = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))
u_star = Exact_u.T.flatten()[:, None]
v_star = Exact_v.T.flatten()[:, None]
h_star = Exact_h.T.flatten()[:, None]

Data loaded successfully!


In [ ]:
# Define domain bounds (de PINNs_Torch)
lb = np.array([-5.0, 0.0])  # Lower bounds [x_min, t_min]
ub = np.array([5.0, np.pi/2])  # Upper bounds [x_max, t_max]

# Number of training points
N0 = 100     # Points for initial condition
N_b = 100    # Points for boundary condition
N_f = 20000  

)
layers = [2, 40, 40, 40, 2]  # [input_dim, hidden_layers..., output_dim]

# Sample initial condition points
idx_x = np.random.choice(x.shape[0], N0, replace=False)
x0 = x[idx_x, :]
u0 = Exact_u[idx_x, 0:1]
v0 = Exact_v[idx_x, 0:1]

# Sample boundary points
idx_t = np.random.choice(t.shape[0], N_b, replace=False)
tb = t[idx_t, :]

# Sample collocation points using Latin Hypercube Sampling
X_f = lb + (ub - lb) * lhs(2, N_f)

# Combine training points
X0 = np.concatenate((x0, 0 * x0), 1)  # (x0, 0)
X_lb = np.concatenate((0 * tb + lb[0], tb), 1)  # (lb[0], tb)
X_ub = np.concatenate((0 * tb + ub[0], tb), 1)  # (ub[0], tb)
X_u_train = np.vstack([X0, X_lb, X_ub])




=== OPTIMIZATION SUMMARY ===
Network architecture: [2, 40, 40, 40, 2]
Collocation points (N_f): 20000
Total parameters: ~3360
Expected speedup: ~2-3x faster than original



In [110]:
# Create the model
model = PhysicsInformedNN(x0, u0, v0, tb, X_f, layers, lb, ub, X_u_train)

# OPTIMIZATION 4: Reduced Adam epochs from 5000 to 3000 (40% reduction)
# Train the model
start_time = time.time()
model.train(epochs=5000)  # Reduced from 5000 to 3000
elapsed = time.time() - start_time
print(f'Training time: {elapsed:.4f} seconds')

Epoch 0, Loss Adam: 3.05390e+00, Time: 0.02s
Epoch 100, Loss Adam: 3.75721e-01, Time: 1.75s
Epoch 100, Loss Adam: 3.75721e-01, Time: 1.75s
Epoch 200, Loss Adam: 2.80456e-01, Time: 1.57s
Epoch 200, Loss Adam: 2.80456e-01, Time: 1.57s
Epoch 300, Loss Adam: 1.68176e-01, Time: 1.38s
Epoch 300, Loss Adam: 1.68176e-01, Time: 1.38s
Epoch 400, Loss Adam: 1.15058e-01, Time: 1.41s
Epoch 400, Loss Adam: 1.15058e-01, Time: 1.41s
Epoch 500, Loss Adam: 9.10290e-02, Time: 1.42s
Epoch 500, Loss Adam: 9.10290e-02, Time: 1.42s
Epoch 600, Loss Adam: 8.07046e-02, Time: 1.71s
Epoch 600, Loss Adam: 8.07046e-02, Time: 1.71s
Epoch 700, Loss Adam: 7.34123e-02, Time: 1.87s
Epoch 700, Loss Adam: 7.34123e-02, Time: 1.87s
Epoch 800, Loss Adam: 6.68138e-02, Time: 1.79s
Epoch 800, Loss Adam: 6.68138e-02, Time: 1.79s
Epoch 900, Loss Adam: 6.15533e-02, Time: 1.77s
Epoch 900, Loss Adam: 6.15533e-02, Time: 1.77s
Epoch 1000, Loss Adam: 5.81656e-02, Time: 1.69s
Epoch 1000, Loss Adam: 5.81656e-02, Time: 1.69s
Epoch 1100, L

In [111]:
# Make predictions
x_star = torch.tensor(X_star[:, 0:1], dtype=torch.float32).to(device)
t_star = torch.tensor(X_star[:, 1:2], dtype=torch.float32).to(device)
with torch.no_grad():
    u_pred_t, v_pred_t = model.net_uv(x_star, t_star)

# Convert to numpy
u_pred = u_pred_t.cpu().numpy()
v_pred = v_pred_t.cpu().numpy()

# For PDE residuals
x_f = torch.tensor(X_star[:, 0:1], dtype=torch.float32).to(device).requires_grad_(True)
t_f = torch.tensor(X_star[:, 1:2], dtype=torch.float32).to(device).requires_grad_(True)
f_u_pred_t, f_v_pred_t = model.net_f_uv(x_f, t_f)

# Detach and convert to numpy
f_u_pred = f_u_pred_t.detach().cpu().numpy()
f_v_pred = f_v_pred_t.detach().cpu().numpy()

# Compute amplitude
h_pred = np.sqrt(u_pred ** 2 + v_pred ** 2)

# Calculate errors
error_u = np.linalg.norm(u_star - u_pred, 2) / np.linalg.norm(u_star, 2)
error_v = np.linalg.norm(v_star - v_pred, 2) / np.linalg.norm(v_star, 2)
error_h = np.linalg.norm(h_star - h_pred, 2) / np.linalg.norm(h_star, 2)

print(f'Error u: {error_u:.3e}')
print(f'Error v: {error_v:.3e}')
print(f'Error h: {error_h:.3e}')

# Reshape predictions for visualization
U_pred = griddata(X_star, u_pred.flatten(), (X, T), method='cubic')
V_pred = griddata(X_star, v_pred.flatten(), (X, T), method='cubic')
H_pred = griddata(X_star, h_pred.flatten(), (X, T), method='cubic')

U_star = griddata(X_star, u_star.flatten(), (X, T), method='cubic')
V_star = griddata(X_star, v_star.flatten(), (X, T), method='cubic')
H_star = griddata(X_star, h_star.flatten(), (X, T), method='cubic')

FU_pred = griddata(X_star, f_u_pred.flatten(), (X, T), method='cubic')
FV_pred = griddata(X_star, f_v_pred.flatten(), (X, T), method='cubic')

Error u: 1.020e-02
Error v: 1.583e-02
Error h: 3.146e-03


In [112]:
# Save results
np.savez_compressed('NLS_PINN_results_optimized.npz',
     x=X[0, :],                  # x-axis
     t=T[:, 0],                  # t-axis
     X=X,
     T=T,
     Exact_h = Exact_h,
     U_star=U_star,
     V_star=V_star,
     H_star=H_star,
     U_pred=U_pred,
     V_pred=V_pred,
     H_pred=H_pred,
     X_u_train=X_u_train,
     X_f=X_f,
     lb=lb,
     ub=ub,
     error_u=error_u,
     error_v=error_v,
     error_h=error_h
)

# Save metadata
metadata = {
     'layers': layers,
     'N0': int(N0),
     'Nb': int(N_b),
     'Nf': int(N_f),
     'lb': lb.tolist(),
     'ub': ub.tolist(),
     'error_u': float(error_u),
     'error_v': float(error_v),
     'error_h': float(error_h),
     'training_time_sec': float(elapsed),
     'framework': 'PyTorch Optimized',
     'device': str(device),
     'architecture': 'Speed-optimized version',
     'optimizations_applied': {
         '1_cuDNN_benchmark': True,
         '2_N_f_reduced': '20000 -> 10000',
         '3_network_architecture': '[2,40,40,40,2] -> [2,32,32,32,2]',
         '4_adam_epochs': '5000 -> 3000',
         'expected_speedup': '2-3x faster'
     }
}

with open('NLS_PINN_metadata_optimized.json', 'w') as f:
    json.dump(metadata, f, indent=4)

print('\n=== Results saved successfully ===')
print(f'Training time: {elapsed:.2f} seconds')
print(f'Architecture: {layers}')
print(f'Errors - u: {error_u:.3e}, v: {error_v:.3e}, h: {error_h:.3e}')



=== Results saved successfully ===
Training time: 280.87 seconds
Architecture: [2, 40, 40, 40, 2]
Errors - u: 1.020e-02, v: 1.583e-02, h: 3.146e-03
